In [29]:
#Required Libraries and package of python
import nltk
import gensim
from gensim.models import Word2Vec
import string
from nltk.corpus import stopwords
from gensim.models import KeyedVectors
from nltk.tokenize import sent_tokenize,word_tokenize
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy import spatial
import spacy
nlp=spacy.load('en_core_web_sm')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

import nltk 
from nltk.tokenize import LineTokenizer 
from nltk.stem import PorterStemmer     
tk = LineTokenizer()   
lm = WordNetLemmatizer() 
from nltk.corpus import wordnet as wn
import re
import string

In [30]:
#Pretrained Word2Vec model. It was pretrained on Google News Corpus
filename = 'GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(filename, binary=True)

In [31]:
#Stopwords
stop_words = set(stopwords.words('english'))
sw=stop_words

In [32]:
#STS-2017 Dataset is used here.Here, we are taking the first sentence and second sentence into two list. 
fh=open('arabic_english_data.txt')
c=0
s1=[]
s2=[]
for line in fh:
    line=line.lower()
    c+=1
    line=line.rstrip()
    #print(c)
    sentences=sent_text = sent_tokenize(line)
    #print(c, sentences)
    s1.append(sentences[0])
    s2.append(sentences[1]) 
print(c)

250


In [33]:
file=open('resultwordcommean1ar.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    s1word=[]
    for word in ns1:
        for c in string.punctuation:
            word=word.replace(c,'')
        if word not in sw:
            s1word.append(lemmatizer.lemmatize(word))
    #print(s1word)
    s2word=[]
    ns2=s2[i].split(' ')
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            s2word.append(lemmatizer.lemmatize(word))
    #print(s2word)
    max_score=[]
    for word in s1word:
        if word in model.wv.vocab:
            score=[]
            for w in s2word:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score.append(max(score))
    #print(max_score)
    
    max_score2=[]
    for word in s2word:
        if word in model.wv.vocab:
            score=[]
            for w in s1word:
                if w in model.wv.vocab:
                    score.append(model.similarity(word,w))
        max_score2.append(max(score))
    #print(max_score2)
    
    count=0
    summ=0
    for val in max_score:
        summ+=val
        count+=1
    average=(summ*5)/count
    
    count1=0
    summ1=0
    for val in max_score2:
        summ1+=val
        count1+=1
    average1=(summ1*5)/count1
    
    aver=(average+average1)/2
    #print('i:',i+1,aver)
    a=str(round(aver,6))+'\n'
    file.writelines(a)
    i+=1
    
file.close()
data1=[]
data2=[]
f1=open('resultwordcommean1ar.txt')
f2=open('STS.gs.track2.ar-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
#print(len(data1),len(data2))
#mse = np.square(np.subtract(data2,data1)).mean()
#print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

Length: 250


<ipython-input-33-1e2ce0dbec48>:25: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.vocab:
<ipython-input-33-1e2ce0dbec48>:28: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if w in model.wv.vocab:
<ipython-input-33-1e2ce0dbec48>:35: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.vocab:
<ipython-input-33-1e2ce0dbec48>:38: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if w in model.wv.vocab:


Pearson: 66.45113259682897 Spearman: 68.84110793004312


In [34]:
file=open('result1ar.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    sum1=np.zeros((300,), dtype="float32")
    for word in ns1:
        for c in string.punctuation:
                word=word.replace(c,"")
        #print(word)
        if word not in sw:
            if(word in model.wv.vocab): 
                sum1=np.add(sum1,model[word])
    average1=np.divide(sum1,len(ns1),dtype="float32")
            
    ns2=s2[i].split(' ')
    sum2=np.zeros((300,),dtype="float32")
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                sum2=np.add(sum2,model[word])
    average2=np.divide(sum2,len(ns2),dtype="float32")
    sim = 1 - spatial.distance.cosine(average1,average2)
    sim=sim*5
    a=str(round(sim,6))+'\n'
    file.writelines(a)
    #print(i,'similarity',sim)
    i+=1 
file.close()


data1=[]
data2=[]
f1=open('result1ar.txt')
f2=open('STS.gs.track2.ar-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

Length: 250
Pearson: 61.85637989337756 Spearman: 65.5204236906536


<ipython-input-34-58a3a58ad988>:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if(word in model.wv.vocab):
<ipython-input-34-58a3a58ad988>:24: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if(word in model.wv.vocab):


In [35]:
file=open('resultNP_VP_compare12ar.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word.strip())
    return sent

def result(s1,s2):
    s1word=[]
    s2word=[]
    for word in s1:
        if word not in sw:
            s1word.append(word)
    for word in s2:
        if word not in sw:
            s2word.append(word)
            
    #print(s1word,s2word)
    if len(s1word)==0 or len(s2word)==0:
        aver=0
    else:
        max_score=[]
        for word in s1word:
            score=[0]
            if word in model.wv.vocab:
                for w in s2word:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            max_score.append(max(score))
        #print(max_score)
    
        max_score2=[]
        for word in s2word:
            score=[0]
            if word in model.wv.vocab:
                for w in s1word:
                    if w in model.wv.vocab:
                        score.append(model.similarity(word,w))
            max_score2.append(max(score))
        #print(max_score2)
    
        average=sum(max_score)/(len(max_score))
    
        average1=sum(max_score2)/(len(max_score2))
    
        aver=(average+average1)/2
    return aver

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    sen1np=listtostr(npp).strip()
    sen1np=sen1np.split(' ')
    #print(sen1np, len(sen1np))
    sen1vp=listtostr(vpp).strip()
    sen1vp=sen1vp.split(' ')
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp).strip()
    sen2np=sen2np.split(' ')
    sen2vp=listtostr(vpp).strip()
    sen2vp=sen2vp.split(' ')
    
    avernp=result(sen1np,sen2np)
    #print('NP:',avernp)
    avervp=result(sen1vp,sen2vp)
    #print('VP:',avervp)
    #if(avernp==0 or avervp==0):
     #   av=(avernp+avervp)*5
    #else:
    av=avernp*4+avervp
    av=str(round(av,6))+'\n'
    #print('No:',i+1,'similarity:',av)
    file.writelines(av)
    
    i=i+1
    
file.close()

data1=[]
data2=[]
f1=open('resultNP_VP_compare12ar.txt')
f2=open('STS.gs.track2.ar-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse=np.square(np.subtract(data2,data1)).mean()
print("MSE:",mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

Length: 250


<ipython-input-35-9fae6c049214>:40: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.vocab:
<ipython-input-35-9fae6c049214>:42: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if w in model.wv.vocab:
<ipython-input-35-9fae6c049214>:50: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if word in model.wv.vocab:
<ipython-input-35-9fae6c049214>:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if w in model.wv.vocab:


250 250
MSE: 0.08375136343654033
Pearson: 58.75364267134348 Spearman: 61.503226607079874


In [36]:
file=open('resultpos1ar.txt','w')

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i].split(' ')
    s1word=[]
    for word in ns1:
        for c in string.punctuation or c.isspace():
            word=word.replace(c,'')
        if word not in sw:
            s1word.append(word)
    #print(i+1,s1word)
    
    s2word=[]
    ns2=s2[i].split(' ')
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            s2word.append(word)
    #print(i+1, s2word)
    tag1=nltk.pos_tag(s1word,tagset='universal')
    tag2=nltk.pos_tag(s2word,tagset='universal')
    #print(tag1,tag2)
    k=0
    max_score=[]
    for k in tag1:
        sscore=[]
        for j in tag2:
            if (k[1]==j[1]):
                if k[0] in model.wv.vocab:
                    if j[0] in model.wv.vocab:
                        a=model.similarity(k[0],j[0])
                        #print(k[0],j[0],a)
                        sscore.append(a)
                    #print(sscore)
        if(len(sscore)>=1):
            max_score.append(max(sscore))
    #print(max_score)
    count=0
    summ=0
    for val in max_score:
        summ+=val
        count+=1
    average=(summ*5)/count
    #print('i:',i+1,average)
    a=str(round(average,6))+'\n'
    file.writelines(a)
    i+=1
file.close()

data1=[]
data2=[]
f1=open('resultpos1ar.txt')
f2=open('STS.gs.track2.ar-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

Length: 250


<ipython-input-36-82ad426102cd>:34: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if k[0] in model.wv.vocab:
<ipython-input-36-82ad426102cd>:35: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if j[0] in model.wv.vocab:


250 250
MSE: 0.08890595816937903
Pearson: 52.29730283673082 Spearman: 52.8396120057849


In [37]:
file=open('newNpVpar.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def vectorvalue(subsent):
    #print(subsent)
    subsent=subsent.split(" ")
    sum1=np.zeros((300,), dtype="float32")
    for word in subsent:
        if word not in sw:
            if(word in model.wv.vocab):
                sum1=np.add(sum1,model[word])
    if len(subsent)>0:
        average1=np.divide(sum1,len(subsent),dtype="float32")
    else:
        average1=sum1
    return average1

def listvalue(pp):
    pplist=[]
    if(len(pp)>0):
        j=0
        while(j<len(pp)):
            pplist.append(vectorvalue(pp[j]))
            j=j+1
    else:
        pplist.append(np.zeros((300,), dtype="float32"))
    return pplist

def score(nplist,np2list,vplist,vp2list,a):
    simlist=[]
    if(np.all(nplist)==0 and np.all(np2list)==0 and np.all(vplist)==0 and np.all(vp2list)==0):
        aa=0
    else:
        for value in nplist:
            npsim=[]
            for value2 in np2list:
                sim=(1 - spatial.distance.cosine(value,value2))*(5/a)
                if( np.isnan(sim)==1):
                    sim=0
                npsim.append(sim)
            #print('np:',max(npsim))
            simlist.append(max(npsim))
        
        for v in vplist:
            vpsim=[]
            for v2 in vp2list:
                sim=(1-spatial.distance.cosine(v,v2))*(5/a)
                if np.isnan(sim)==1:
                    sim=0
                vpsim.append(sim)
            #print('vp:',max(vpsim))
            simlist.append(max(vpsim))
        if(len(simlist)>0):    
            bb=max(simlist)
            p=0
            while(p<len(simlist)):
                if simlist[p]<bb/2:
                    simlist[p]=0
                p=p+1
        aa = sum(simlist)
    return aa

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    nplist=[]
    vplist=[]
    np2list=[]
    vp2list=[]
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    nplist=listvalue(npp)
    vplist=listvalue(vpp)
    a=len(npp)+len(vpp)
    
    ns2=s2[i]
    #print(ns2)
    npp2,vpp2=partitioning(ns2)
    #print(npp2,vpp2)
    np2list=listvalue(npp2)
    vp2list=listvalue(vpp2)
    b=len(npp2)+len(vpp2)
    
    if a>=b:
        sim=score(nplist,np2list,vplist,vp2list,a)
    else:
        sim=score(np2list,nplist,vp2list,vplist,b)
    #print(i+1,'similarity:',sim)
    sim=str(round(sim,2))+'\n'
    file.writelines(sim)
    i=i+1
    
file.close()

data1=[]
data2=[]
f2=open('STS.gs.track2.ar-en.txt')
f1=open('newNpVpar.txt')
#f2=open('newNpVp.txt')

def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
#print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

Length: 250


<ipython-input-37-944ed4b6d970>:23: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if(word in model.wv.vocab):


250 250
Pearson: 62.739439390633 Spearman: 63.92569566511496


In [38]:
file=open('new_vector_formar.txt','w')
ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<250):
    ns1=s1[i].split(' ')
    sum1=np.zeros((300,), dtype="float32")
    w=0
    for word in ns1:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                if w==0:
                    sum1=np.add(sum1,model[word])
                    w=w+1
                else:
                    av=sum(sum1)/300
                    a=list(model[word])
                    sum1=list(sum1)
                    k=0
                    while(k<300):
                        if abs(a[k]-sum1[k])>=0.16:   #works better at 0.135
                            sum1[k]=a[k]+sum1[k]
                            k=k+1
                        else:
                            sum1[k]=sum1[k]
                            k=k+1
                    sum1=np.asarray(sum1)
    average1=np.divide(sum1,1,dtype="float32")
            
    ns2=s2[i].split(' ')
    sum2=np.zeros((300,),dtype="float32")
    
    w=0
    for word in ns2:
        for c in string.punctuation:
                word=word.replace(c,"")
        if word not in sw:
            if(word in model.wv.vocab):
                if w==0:
                    sum2=np.add(sum2,model[word])
                    w=w+1
                else:
                    av=sum(sum2)/300
                    a=list(model[word])
                    sum2=list(sum2)
                    k=0
                    while(k<300):
                        if abs(a[k]-sum2[k])>=0.16:    # better at 0.16--- 73.17
                            sum2[k]=a[k]+sum2[k]
                            k=k+1
                        else:
                            sum2[k]=sum2[k]
                            k=k+1
                    sum2=np.asarray(sum2)        
    average2=np.divide(sum2,1,dtype="float32")

    sim = 1 - spatial.distance.cosine(average1,average2)
    sim=sim*5
    a=str(round(sim,6))+'\n'
    file.writelines(a)
    #print(i,'similarity',sim)
    i+=1
file.close()


data1=[]
data2=[]
f1=open('new_vector_formar.txt')
f2=open('STS.gs.track2.ar-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)

Length: 250


<ipython-input-38-df12a24ee299>:14: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if(word in model.wv.vocab):
<ipython-input-38-df12a24ee299>:41: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if(word in model.wv.vocab):


Pearson: 67.82589916042335 Spearman: 67.96456310723458


In [39]:
file=open('resultNP_VP12ar.txt','w')

def partitioning(sentence):
    sen=''
    for word in sentence:
        for c in string.punctuation:
            word=word.replace(c,"")
        sen+=word
    sentence=sen
    doc=nlp(sen)
    nounp=[]
    verbp=[]
    nounp=[chunk.text for chunk in doc.noun_chunks]
    verbp=[token.lemma_ for token in doc if token.pos_ == "VERB"]
    return nounp,verbp

def vectorvalue(subsent):
    #print(subsent)
    subsent=subsent.split(" ")
    sum1=np.zeros((300,), dtype="float32")
    for word in subsent:
        if word not in sw:
            if(word in model.wv.vocab):
                sum1=np.add(sum1,model[word])
    if len(subsent)>0:
        average1=np.divide(sum1,len(subsent),dtype="float32")
    else:
        average1=sum1
    return average1

def listtostr(phrase):
    sent=''
    for word in phrase:
        sent=sent+" "+str(word)
    return vectorvalue(sent)

ns1=[]
ns2=[]
i=0
print('Length:',len(s1))
while(i<len(s1)):
    ns1=s1[i]
    #print(ns1)
    npp,vpp=partitioning(ns1)
    #print(npp,vpp)
    sen1np=listtostr(npp)
    sen1vp=listtostr(vpp)
        
    ns2=s2[i]
    #print(ns2)
    npp,vpp=partitioning(ns2)
    #print(npp,vpp)
    sen2np=listtostr(npp)
    sen2vp=listtostr(vpp)
        
    if ((np.all(sen1np==0)) or (np.all(sen2np==0))):
            nppsim=0
    else:
        nppsim= 1 - spatial.distance.cosine(sen1np,sen2np)
    #print('nounphase:',nppsim)
    if ((np.all(sen1vp==0)) or (np.all(sen2vp==0))):
            vppsim=0
    else:
        vppsim=1 - spatial.distance.cosine(sen1vp,sen2vp)
    similarity=nppsim*4+vppsim
    sim=str(round(similarity,2))+'\n'
    file.writelines(sim)
    i+=1
    
file.close()

data1=[]
data2=[]
f1=open('resultNP_VP12ar.txt')
f2=open('STS.gs.track2.ar-en.txt')
def listvalue(file):
    value=[]
    for line in file:
        value.append(float(line)/5)
    return value
data1=listvalue(f1)
data2=listvalue(f2)
print(len(data1),len(data2))
mse = np.square(np.subtract(data2,data1)).mean()
print('MSE:',mse)
corr,_=pearsonr(data1,data2)
scor,_=spearmanr(data1,data2)
print('Pearson:',corr*100,'Spearman:',scor*100)
#P=67.51, S=70.18

Length: 250


<ipython-input-39-8f2113a9003e>:23: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if(word in model.wv.vocab):


250 250
MSE: 0.10011280000000002
Pearson: 57.036624541751465 Spearman: 61.221899624587316
